In [1]:
import sys
sys.path.append("/home/wangquanjun/project/LLM_project/starchain/llm_server/chatglm")
from chatglm_extension import ChatGLM

sys.path.append("/home/wangquanjun/project/LLM_project/starchain/llm_server/utils")
from utils import remove_blank_keep_newline
from utils import generate_dirname


from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredWordDocumentLoader
from langchain.schema.document import Document

import os
import time
import datetime
import json
import pandas as pd


In [2]:
from langchain.prompts import PromptTemplate

In [ ]:
class ResumeNormalizer:
    
    def __init__(self, path, num=500):
        self.path = path
        loader = UnstructuredWordDocumentLoader(path)
        docs = loader.load()
        self.context_1 = docs[0].page_content[:num]
        self.context_2 = docs[0].page_content[num:]
        
        self.prompt_template = """
        Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
        """
        
        

In [3]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""

In [3]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""

In [4]:
PROMPT = PromptTemplate.from_template(prompt_template)

In [5]:
PROMPT.input_variables

['context', 'question']

In [6]:
options = ["姓名", "性别", "年龄", "民族", "出生日期", "地址", "学历", "专业", "邮箱", "联系方式"]

In [7]:
option_str = ','.join(options)

In [8]:
question = f"这个应聘者的{option_str}是什么，请以json形式返回结果，最多不超过250字，如果查找不到，则返回'未提供'"

In [9]:
question

"这个应聘者的姓名,性别,年龄,民族,出生日期,地址,学历,专业,邮箱,联系方式是什么，请以json形式返回结果，最多不超过250字，如果查找不到，则返回'未提供'"

In [10]:
path = '/home/wangquanjun/jupyter/app_test/01.ResumeFilter/data/三人行简历-佳晟-熔铸-王仁杰.doc'

In [11]:
loader = UnstructuredWordDocumentLoader(path)

In [12]:
docs = loader.load()

In [13]:
text = docs[0].page_content[:500]

In [14]:
page_content = remove_blank_keep_newline(text)

In [15]:
doc_related = Document(page_content=page_content, metadata={'source': path})

In [16]:
prompt = PROMPT.format(context=doc_related.page_content, question=question)
print(prompt)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

三人行简历

基本资料： 

姓 名：王仁杰 性 别： 男 

身份证号码：51372219881208391X 民 族： 汉

出生年月：1988/12/08 健康状况：良好 

户籍地：四川省成都市 面貌：中共党员 

学 历： 本 科 专业：金属材料工程

地址：四川省成都市温江区 

邮 箱: wang_ren_jie_sc@163.com 联系电话：13568942066 

教育背景： 

2009.09 ~2013.07……西安工业大学北方信息工程学院…金属材料工程…学 士 

毕业设计：海洋性气候对7475铝合金力学性能的影响

工作经历： 

Question: 这个应聘者的姓名,性别,年龄,民族,出生日期,地址,学历,专业,邮箱,联系方式是什么，请以json形式返回结果，最多不超过250字，如果查找不到，则返回'未提供'
Helpful Answer:


In [17]:
llm = ChatGLM()

In [18]:
llm.endpoint_url = 'http://0.0.0.0:4002'

In [19]:
text = ""
for chunk in llm.stream(prompt):
    text += chunk
    print(chunk, end="", flush=True)

 {
"姓名": "王仁杰"，
"性别": "男"，
"年龄": "33"，
"民族": "汉"，
"出生日期": "1988/12/08"，
"地址": "四川省成都市温江区"，
"学历": "本科"，
"专业": "金属材料工程"，
"邮箱": "wang_ren_jie_sc@163.com"，
"联系方式": "13568942066"
}